In [1]:
import cv2
path = 'mats/pedestrian/pedestrian/input'
for i in range (300 ,1100) :
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  cv2.imshow ( " I " ,I )
  cv2.waitKey(10)

cv2.destroyAllWindows()

In [2]:
# Substract the frames
import cv2
path = 'mats/pedestrian/pedestrian/input'

prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in000300.jpg ' ), cv2.COLOR_BGR2GRAY)

for i in range (300 ,1100,2):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  cv2.imshow ( " diff " ,diff )
  cv2.waitKey(10)
  prev_frame = gray

cv2.destroyAllWindows()

In [3]:
# binarize the difference

import cv2
path = 'mats/pedestrian/pedestrian/input'

prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in000300.jpg ' ), cv2.COLOR_BGR2GRAY)

for i in range (300 ,1100,2):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  T,binarized = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
  cv2.imshow ( " binarized " ,binarized )
  cv2.waitKey(10)
  prev_frame = gray
  
cv2.destroyAllWindows()

In [4]:
# on binarized image, perform median filtration, then erosion and dilation

import cv2
path = 'mats/pedestrian/pedestrian/input'

prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in000300.jpg ' ), cv2.COLOR_BGR2GRAY)

for i in range (300 ,1100,2):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  T,binarized = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
  
  median = cv2.medianBlur(binarized, 5)
  kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  kernel_2 = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))

  eroded = cv2.erode(median, kernel_1, iterations=1)
  dilated = cv2.dilate(eroded, kernel_1, iterations=1)

  for _ in range(4):
    dilated = cv2.dilate(dilated, kernel_2, iterations=1)
    eroded = cv2.erode(dilated, kernel_2, iterations=1)


  cv2.imshow ( " dilated " ,eroded )
  cv2.waitKey(10)
  prev_frame = gray
  
cv2.destroyAllWindows()

In [5]:
# Indexation and analysis
import cv2
import numpy as np
path = 'mats/pedestrian/pedestrian/input'

prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in000300.jpg ' ), cv2.COLOR_BGR2GRAY)

for i in range (300 ,1100):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  T,binarized = cv2.threshold(diff, 10, 255, cv2.THRESH_BINARY)
  
  median = cv2.medianBlur(binarized, 5)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))

  eroded = cv2.erode(median, kernel, iterations=1)
  dilated = cv2.dilate(eroded, kernel, iterations=1)

  retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

  I_VIS = I.copy()
  if(stats.shape[0]>1):
    tab = stats[1:,4]
    pi = np.argmax(tab)

    if (tab[pi]>500):
      pi = pi + 1
      cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
      cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))



  cv2.imshow ( " I_VIS " ,I_VIS )







  cv2.waitKey(10)
  prev_frame = gray
  
cv2.destroyAllWindows()

In [6]:
#Define precision, recall and F1-score

#Define global TP TN, FP, FN
TP , TN , FP , FN = 0 , 0 , 0 , 0
import cv2
import numpy as np

path = 'mats/pedestrian/pedestrian/input'
path_gt = 'mats/pedestrian/pedestrian/groundtruth'
prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in000300.jpg ' ), cv2.COLOR_BGR2GRAY)
with open ( f'mats/pedestrian/pedestrian/temporalROI.txt ' , 'r' ) as f:
  line = f.readline()
  a,b = line.split()
  roi_start = int(a)
  roi_end = int(b)

results_pedestrian = ''

for i in range (roi_start ,roi_end):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  I_GT=cv2.imread ( f'{path_gt}/gt%06d.png ' % i )

  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  T,binarized = cv2.threshold(diff, 7, 255, cv2.THRESH_BINARY)
  
  median = cv2.medianBlur(binarized, 5)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))

  eroded = cv2.erode(median, kernel, iterations=1)
  dilated = cv2.dilate(eroded, kernel, iterations=1)

  retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

  # cv2.imshow ( " dilated " ,np.uint8(labels * 255 / retval) )

  I_VIS = I.copy()
  if(stats.shape[0]>1):
    tab = stats[1:,4]
    pi = np.argmax(tab)
    # print(f'max area = {tab[pi]}')

    if (tab[pi]>500):
      pi = pi + 1
      cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
      cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


      TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
      TP_S = np . sum ( TP_M )
      TP = TP + TP_S

      FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
      FP_S = np . sum ( FP_M )
      FP = FP + FP_S

      FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
      FN_S = np . sum ( FN_M )
      FN = FN + FN_S

      TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
      TN_S = np . sum ( TN_M )
      TN = TN + TN_S

      if TP + FP != 0:
        precision = TP / (TP + FP)
      else:
        precision = 0
      if TP + FN != 0:
        recall = TP / (TP + FN)
      else:
        recall = 0
      if precision + recall != 0:
        F1 = 2 * precision * recall / (precision + recall)
      else:
        F1 = 0
    else:
      precision = 0
      recall = 0
      F1 = 0
  else:
    precision = 0
    recall = 0
    F1 = 0


  # add to results_pedestrian json
  results_pedestrian = results_pedestrian + f'{{"frame": {i}, "precision": {precision:.2f}, "recall": {recall:.2f}, "F1": {F1:.2f}}},'
  cv2.imshow ( " I_VIS " ,I_VIS )
  cv2.waitKey(10)
  prev_frame = gray
  
cv2.destroyAllWindows()

import json
with open('results_pedestrian.json', 'w') as f:
  json.dump(results_pedestrian, f)

In [7]:
# For highway dataset
TP , TN , FP , FN = 0 , 0 , 0 , 0
import cv2
import numpy as np

path = 'mats/highway/highway/input'
path_gt = 'mats/highway/highway/groundtruth'
with open ( f'mats/highway/highway/temporalROI.txt ' , 'r' ) as f:
  line = f.readline()
  a,b = line.split()
  roi_start = int(a)
  roi_end = int(b)

results_highway = ''

prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in%06d.jpg '% roi_start ), cv2.COLOR_BGR2GRAY)

for i in range (roi_start ,roi_end):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  I_GT=cv2.imread ( f'{path_gt}/gt%06d.png ' % i )

  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  T,binarized = cv2.threshold(diff, 10, 255, cv2.THRESH_BINARY)
  
  median = cv2.medianBlur(binarized, 5)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))

  eroded = cv2.erode(median, kernel, iterations=1)
  dilated = cv2.dilate(eroded, kernel, iterations=1)

  retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

  # cv2.imshow ( " dilated " ,np.uint8(labels * 255 / retval) )

  I_VIS = I.copy()
  if(stats.shape[0]>1):
    tab = stats[1:,4]
    pi = np.argmax(tab)
    # print(f'max area = {tab[pi]}')

    if (tab[pi]>300):
      pi = pi + 1
      cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
      cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


  TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
  TP_S = np . sum ( TP_M )
  TP = TP + TP_S

  FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
  FP_S = np . sum ( FP_M )
  FP = FP + FP_S

  FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
  FN_S = np . sum ( FN_M )
  FN = FN + FN_S

  TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
  TN_S = np . sum ( TN_M )
  TN = TN + TN_S

  if TP + FP != 0:
    precision = TP / (TP + FP)
  else:
    precision = 0
  if TP + FN != 0:
    recall = TP / (TP + FN)
  else:
    recall = 0
  if precision + recall != 0:
    F1 = 2 * precision * recall / (precision + recall)
  else:
    F1 = 0


  # add to results_pedestrian json
  results_highway = results_pedestrian + f'{{"frame": {i}, "precision": {precision:.2f}, "recall": {recall:.2f}, "F1": {F1:.2f}}},'
  cv2.imshow ( " I_VIS " ,I_VIS )
  cv2.waitKey(10)
  prev_frame = gray
  
cv2.destroyAllWindows()

import json
with open('results_highway.json', 'w') as f:
  json.dump(results_highway, f)

In [8]:
# For office dataset
TP , TN , FP , FN = 0 , 0 , 0 , 0
import cv2
import numpy as np

path = 'mats/office/office/input'
path_gt = 'mats/office/office/groundtruth'
prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in000300.jpg ' ), cv2.COLOR_BGR2GRAY)
with open ( f'mats/office/office/temporalROI.txt ' , 'r' ) as f:
  line = f.readline()
  a,b = line.split()
  roi_start = int(a)
  roi_end = int(b)

results_office = ''

for i in range (roi_start ,roi_end):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  I_GT=cv2.imread ( f'{path_gt}/gt%06d.png ' % i )

  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  T,binarized = cv2.threshold(diff, 5, 255, cv2.THRESH_BINARY)
  
  median = cv2.medianBlur(binarized, 5)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))

  eroded = cv2.erode(median, kernel, iterations=1)
  dilated = cv2.dilate(eroded, kernel, iterations=1)


  for _ in range(10):
    dilated = cv2.dilate(eroded, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)


  retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

  # cv2.imshow ( " dilated " ,np.uint8(labels * 255 / retval) )

  I_VIS = I.copy()
  if(stats.shape[0]>1):
    tab = stats[1:,4]
    pi = np.argmax(tab)
    # print(f'max area = {tab[pi]}')

    if (tab[pi]>500):
      pi = pi + 1
      cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
      cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


  TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
  TP_S = np . sum ( TP_M )
  TP = TP + TP_S

  FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
  FP_S = np . sum ( FP_M )
  FP = FP + FP_S

  FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
  FN_S = np . sum ( FN_M )
  FN = FN + FN_S

  TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
  TN_S = np . sum ( TN_M )
  TN = TN + TN_S

  if TP + FP != 0:
    precision = TP / (TP + FP)
  else:
    precision = 0
  if TP + FN != 0:
    recall = TP / (TP + FN)
  else:
    recall = 0
  if precision + recall != 0:
    F1 = 2 * precision * recall / (precision + recall)
  else:
    F1 = 0

  # add to results_pedestrian json
  results_office = results_office + f'{{"frame": {i}, "precision": {precision:.2f}, "recall": {recall:.2f}, "F1": {F1:.2f}}},'
  cv2.imshow ( " I_VIS " ,I_VIS )
  cv2.waitKey(10)
  prev_frame = gray
  
cv2.destroyAllWindows()

import json
with open('results_office.json', 'w') as f:
  json.dump(results_office, f)

In [9]:
### Modify highway to detect all the vehicles

# For highway dataset
TP , TN , FP , FN = 0 , 0 , 0 , 0
import cv2
import numpy as np

path = 'mats/highway/highway/input'
path_gt = 'mats/highway/highway/groundtruth'
prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in000300.jpg ' ), cv2.COLOR_BGR2GRAY)
with open ( f'mats/highway/highway/temporalROI.txt ' , 'r' ) as f:
  line = f.readline()
  a,b = line.split()
  roi_start = int(a)
  roi_end = int(b)

results_highway = ''

for i in range (roi_start ,roi_end):
  I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
  I_GT=cv2.imread ( f'{path_gt}/gt%06d.png ' % i )

  gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  diff = cv2.absdiff(gray, prev_frame)
  T,binarized = cv2.threshold(diff, 10, 255, cv2.THRESH_BINARY)
  
  median = cv2.medianBlur(binarized, 5)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

  eroded = cv2.erode(median, kernel, iterations=1)
  dilated = cv2.dilate(eroded, kernel, iterations=1)

  retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)


  I_VIS = I.copy()
  if(stats.shape[0]>1):
    tab = stats[1:,4]
    pi = np.argmax(tab)

    for pi in range(1, stats.shape[0]):
        if (stats[pi,4]>300):
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))



  TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
  TP_S = np . sum ( TP_M )
  TP = TP + TP_S

  FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
  FP_S = np . sum ( FP_M )
  FP = FP + FP_S

  FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
  FN_S = np . sum ( FN_M )
  FN = FN + FN_S

  TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
  TN_S = np . sum ( TN_M )
  TN = TN + TN_S

  if TP + FP != 0:
    precision = TP / (TP + FP)
  else:
    precision = 0
  if TP + FN != 0:
    recall = TP / (TP + FN)
  else:
    recall = 0
  if precision + recall != 0:
    F1 = 2 * precision * recall / (precision + recall)
  else:
    F1 = 0

  print(f'precision = {precision} recall = {recall} F1 = {F1}')

  # add to results_pedestrian json
  results_highway = results_pedestrian + f'{{"frame": {i}, "precision": {precision:.2f}, "recall": {recall:.2f}, "F1": {F1:.2f}}},'
  cv2.imshow ( " I_VIS " ,I_VIS )
  cv2.waitKey(10)
  prev_frame = gray
  
cv2.destroyAllWindows()

import json
with open('results_highway.json', 'w') as f:
  json.dump(results_highway, f)

precision = 0.005873976504093984 recall = 1.0 F1 = 0.011679348787825164
precision = 0.006830025501718594 recall = 0.9166666666666666 F1 = 0.01355902357421144
precision = 0.006825031022868286 recall = 0.8461538461538461 F1 = 0.013540842345906972
precision = 0.006823216659282233 recall = 0.7938144329896907 F1 = 0.013530135301353014
precision = 0.006998272578287638 recall = 0.7452830188679245 F1 = 0.013866339023212954
precision = 0.007042253521126761 recall = 0.6794871794871795 F1 = 0.013940031562335614
precision = 0.007106801275239107 recall = 0.6233009708737864 F1 = 0.014053367773570037
precision = 0.007123421011879742 recall = 0.5590277777777778 F1 = 0.014067585574171563
precision = 0.0071625953354703216 recall = 0.5070422535211268 F1 = 0.014125648515498974
precision = 0.007222369467267426 recall = 0.4678111587982833 F1 = 0.014225122349102772
precision = 0.007353915463097919 recall = 0.43302990897269183 F1 = 0.014462226661744586
precision = 0.007855078219809802 recall = 0.4139534883720

In [10]:
### Modify highway to detect all the vehicles

# For highway dataset
global TP , TN , FP , FN
import cv2
import numpy as np


def analyse_video(_name, mask_size, threshold, erosion, dilation, min_area):
    TP , TN , FP , FN = 0 , 0 , 0 , 0
    path = f'mats/{_name}/{_name}/input'
    path_gt = f'mats/{_name}/{_name}/groundtruth'
    with open ( f'mats/{_name}/{_name}/temporalROI.txt ' , 'r' ) as f:
        line = f.readline()
        a,b = line.split()
        roi_start = int(a)
        roi_end = int(b)

    prev_frame = cv2.cvtColor(cv2.imread ( f'{path}/in%06d.jpg '% roi_start ), cv2.COLOR_BGR2GRAY)

    results = ''

    for i in range (roi_start ,roi_end):
        I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
        I_GT=cv2.imread ( f'{path_gt}/gt%06d.png ' % i )
        # print(f'frame {i}, I = {I.shape}, I_GT = {I_GT.shape}')
        gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(gray, prev_frame)
        T,binarized = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        
        median = cv2.medianBlur(binarized, 5)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (mask_size, mask_size))

        eroded = cv2.erode(median, kernel, iterations=1)
        dilated = cv2.dilate(eroded, kernel, iterations=1)

        retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

        I_VIS = I.copy()
        if(stats.shape[0]>1):
            tab = stats[1:,4]
            pi = np.argmax(tab)

            for pi in range(1, stats.shape[0]):
                if (stats[pi,4]>min_area):
                    cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
                    cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))



                    TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
                    TP_S = np . sum ( TP_M )
                    TP = TP + TP_S

                    FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
                    FP_S = np . sum ( FP_M )
                    FP = FP + FP_S

                    FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
                    FN_S = np . sum ( FN_M )
                    FN = FN + FN_S

                    TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
                    TN_S = np . sum ( TN_M )
                    TN = TN + TN_S

                    if TP + FP != 0:
                        precision = TP / (TP + FP)
                    else:
                        precision = 0
                    if TP + FN != 0:
                        recall = TP / (TP + FN)
                    else:
                        recall = 0
                    if precision + recall != 0:
                        F1 = 2 * precision * recall / (precision + recall)
                    else:
                        F1 = 0
                else:
                    precision = 0
                    recall = 0
                    F1 = 0
            else:
                precision = 0
                recall = 0
                F1 = 0

        cv2.imshow ( " I_VIS " ,I_VIS )
        cv2.waitKey(10)
        prev_frame = gray
    
    cv2.destroyAllWindows()

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    print(f'precision = {precision} recall = {recall} F1 = {F1}')
    
    import json
    with open('results_{_name}.json', 'w') as f:
        json.dump(results, f)

In [11]:
analyse_video('highway', 3, 10, 1, 1, 300)

precision = 0.9378033500983373 recall = 0.6935461669602411 F1 = 0.797388802365177


In [12]:
analyse_video('pedestrian', 2, 9, 1, 1, 400)

precision = 0.5862951986467477 recall = 0.7134964117601666 F1 = 0.6436716733934127
